In [1]:
from collections import namedtuple
import math
from DirectionCoefficient import get_nu_H as DC_get_nu_H
from DirectionCoefficient import get_nu_C as DC_get_nu_C

# 戸建て住宅の外皮性能の簡易評価法（デフォルト面積検討用）

本方法は、建築研究所で公表している戸建て住宅の外皮性能の簡易評価のデフォルト面積を決めるために開発されたモデルである。  
本評価方法をもとに、デフォルト面積を決めるための住宅形状が検討された。  
建築物省エネ法で運用されている外皮性能の簡易評価法と本評価方法は異なることに留意されたい。  

## 1. 検討の流れ

1. 戸建て住宅の外皮性能の簡易評価法（デフォルト面積検討用）を開発
1. 上記の評価法を用いて、住宅形状に関する諸元（階数・床面積等）を決定
1. 決定された住宅形状をもとに、外皮部位の面積・長さ（デフォルト値）を算出
1. 外皮部位の面積・長さ（デフォルト値）から簡易法を作成

ここで示す方法は、上記の1.の評価法である。

## 2. 型

### 2.1 Orientation

主方位から時計回りに0°、90°、180°、270°を表す。

In [2]:
Orientation = namedtuple('Orientation', 'deg0 deg90 deg180 deg270')

## 3. 各関数

### 3.1 外皮平均熱貫流率 (U<sub>A</sub>値)

$$
U_A = \frac{q_{total}}{A_{evp,total}}
$$

$U_A$：外皮平均熱貫流率(W/m<sup>2</sup>K)  
$q_{total}$：単位温度差当たりの外皮熱損失量(W/K)  
$A_{env,total}$：部位の面積の合計(m<sup>2</sup>)  

In [3]:
def get_U_A(q_total, A_env_total):
    return q_total / A_env_total

### 3.2 平均日射熱取得率 ($\eta_{A,H}$値、$\eta_{A,C}$値)

$$
\eta_{A,H}=\frac{m_H}{A_{evp,total}} \times 100
$$

$$
\eta_{A,C}=\frac{m_C}{A_{evp,total}} \times 100
$$

$\eta_{A,H}$：平均日射熱取得率(暖房期)(%)  
$\eta_{A,C}$：平均日射熱取得率(冷房期)(%)  
$m_H$：単位日射強度当たりの暖房期の日射熱取得量(W/(W/m<sup>2</sup>))  
$m_C$：単位日射強度当たりの冷房期の日射熱取得量(W/(W/m<sup>2</sup>))  
$A_{env,total}$：部位の面積の合計(m<sup>2</sup>)  

In [4]:
def get_eta_A_H(m_H, A_env_total):
    return m_H / A_env_total * 100

In [5]:
def get_eta_A_C(m_C, A_env_total):
    return m_C / A_env_total * 100

### 3.3 単位温度差当たりの外皮熱損失量

#### 1) 合計

$$
q_{total} = q_{roof} + q_{window} + q_{wall} + q_{floor} + q_{base}
$$

$q_{total}$：単位温度差当たりの外皮熱損失量(W/K)  
$q_{roof}$：天井又は外気屋根の単位温度差当たりの外皮熱損失量(W/K)  
$q_{window}$：窓の単位温度差当たりの外皮熱損失量(W/K)  
$q_{wall}$：壁の単位温度差当たりの外皮熱損失量(W/K)  
$q_{floor}$：外気床の単位温度差当たりの外皮熱損失量(W/K)  
$q_{base}$：土間床外周部および基礎等単位温度差当たりの外皮熱損失量(W/K)

In [6]:
def get_q_total(q_roof, q_window, q_wall, q_floor, q_base):
    return q_roof + q_window + q_wall + q_floor + q_base

#### 2) 天井又は外気屋根

$$
q_{roof} = \sum_i \left ( A_{roof,i} \times U_{roof} \times H_{roof} \right )
$$

$A_{roof,i}$：i階の天井又は外気屋根の面積の合計(m<sup>2</sup>)  
$U_{roof}$：天井又は外気屋根の熱貫流率(W/m<sup>2</sup>K)  
$H_{roof}$：天井又は外気屋根の温度差係数($=1.0$)  

In [7]:
def get_q_roof(A_roof, U_roof):
    H_roof = 1.0
    return sum( [ A_roof_i * U_roof * H_roof for A_roof_i in A_roof ] )

#### 3) 窓

$$
q_{window} = A_{wnd,0} \times U_{wnd,0} \times H_{wnd} + A_{wnd,90} \times U_{wnd,90} \times H_{wnd} + A_{wnd,180} \times U_{wnd,180} \times H_{wnd} + A_{wnd,270} \times U_{wnd,270} \times H_{wnd}
$$

$$
U_{wnd,0} = U_{wnd,main}
$$

$$
U_{wnd,90} = U_{wnd,other}
$$

$$
U_{wnd,180} = U_{wnd,other}
$$

$$
U_{wnd,270} = U_{wnd,other}
$$

$A_{wnd,0}$：主開口方位の窓面積の合計(m<sup>2</sup>)  
$A_{wnd,90}$：主開口方位から時計回りに90度の方位の窓面積の合計(m<sup>2</sup>)  
$A_{wnd,180}$：主開口方位から時計回りに180度の方位の窓面積の合計(m<sup>2</sup>)  
$A_{wnd,270}$：主開口方位から時計回りに270度の方位の窓面積の合計(m<sup>2</sup>)  
$U_{wnd,0}$：主開口方位の窓の熱貫流率(W/m<sup>2</sup>K)  
$U_{wnd,90}$：主開口方位から時計回りに90度の方位の窓の熱貫流率(W/m<sup>2</sup>K)  
$U_{wnd,180}$：主開口方位から時計回りに180度の方位の窓の熱貫流率(W/m<sup>2</sup>K)  
$U_{wnd,270}$：主開口方位から時計回りに270度の方位の窓の熱貫流率(W/m<sup>2</sup>K)  
$H_{wnd}$：窓の温度差係数($=1.0$)  
$U_{wnd,main}$：主開口方位の窓の熱貫流率(W/m<sup>2</sup>K)  
$U_{wnd,other}$：主開口方位以外の窓の熱貫流率(W/m<sup>2</sup>K)  

In [8]:
def get_q_window(A_wnd, U_wnd_main, U_wnd_other):
    U_wnd = Orientation( deg0 = U_wnd_main, deg90 = U_wnd_other, deg180 = U_wnd_other, deg270 = U_wnd_other )
    H_wnd = 1.0
    return   A_wnd.deg0   * U_wnd.deg0   * H_wnd \
           + A_wnd.deg90  * U_wnd.deg90  * H_wnd \
           + A_wnd.deg180 * U_wnd.deg180 * H_wnd \
           + A_wnd.deg270 * U_wnd.deg270 * H_wnd 

#### 4) 壁

$$
q_{wall} = A_{wall,0} \times U_{wall} \times H_{wall} + A_{wall,90} \times U_{wall} \times H_{wall} + A_{wall,180} \times U_{wall} \times H_{wall} + A_{wall,270} \times U_{wall} \times H_{wall}
$$

$A_{wall,0}$：主開口方位の壁面積の合計(m<sup>2</sup>)  
$A_{wall,90}$：主開口方位から時計回りに90度の方位の壁面積の合計(m<sup>2</sup>)  
$A_{wall,180}$：主開口方位から時計回りに180度の方位の壁面積の合計(m<sup>2</sup>)  
$A_{wall,270}$：主開口方位から時計回りに270度の方位の壁面積の合計(m<sup>2</sup>)  
$U_{wall}$：壁の熱貫流率(W/m<sup>2</sup>K)  
$H_{wall}$：壁の温度差係数($=1.0$)  

In [9]:
def get_q_wall(A_wall, U_wall):
    H_wall = 1.0
    return   A_wall.deg0   * U_wall * H_wall \
           + A_wall.deg90  * U_wall * H_wall \
           + A_wall.deg180 * U_wall * H_wall \
           + A_wall.deg270 * U_wall * H_wall

#### 5) 外気床

$$
q_{floor} = \sum_{i=1} \left ( A_{exfloor,i} \times U_{exfloor,i} \times H_{exfloor,i} \right )
$$

$$
U_{exfloor,i} = \left \{
\begin{array}{ll}
    U_{exfloor,C} & (i=1) \\
    U_{exfloor,O} & (i\ne1)
\end{array}
\right.
$$

$$
H_{exfloor,i} = \left \{
\begin{array}{ll}
    H_{exfloor,C} & (i=1) \\
    H_{exfloor,O} & (i\ne1)
\end{array}
\right.
$$

$A_{exfloor,i}$：i階の外気床の面積の合計(m<sup>2</sup>)  
$U_{exfloor,C}$：床下に面する外気床の熱貫流率(W/m<sup>2</sup>K)  
$U_{exfloor,O}$：外気に面する外気床の熱貫流率(W/m<sup>2</sup>K)  
$H_{exfloor,C}$：床下に面する外気床の温度差係数($=0.7$)  
$H_{exfloor,O}$：外気に面する外気床の温度差係数($=1.0$)  

In [10]:
def get_q_floor(A_exfloor, U_exfloor_C, U_exfloor_O):
    H_exfloor_C = 0.7
    H_exfloor_O = 1.0
    q_floor = 0.0
    for (i, A_exfloor_i) in enumerate(A_exfloor):
        if i == 0:
            q_floor = q_floor + A_exfloor_i * U_exfloor_C * H_exfloor_C
        else:
            q_floor = q_floor + A_exfloor_i * U_exfloor_O * H_exfloor_O
    return q_floor

#### 6) 土間床外周部及び基礎等

$$
q_{base} = L_{base,ex} \times \psi_{base,ex} \times H_{base,ex} + L_{base,in} \times \psi_{base,in} \times H_{base,in}
$$

$L_{base,ex}$：土間床外周部及び基礎等の外気側の長さ(m)  
$L_{base,in}$：土間床外周部及び基礎等の室内側の長さ(m)  
$\psi_{base,ex}$：土間床外周部及び基礎等の外気側の線熱貫流率(W/mK)  
$\psi_{base,in}$：土間床外周部及び基礎等の室内側の線熱貫流率(W/mK)  
$H_{base,ex}$：土間床外周部及び基礎等の外気側の温度差係数($=1.0$)  
$H_{base,in}$：土間床外周部及び基礎等の室内側の温度差係数($=0.7$)

In [11]:
def get_q_base(L_base_ex, L_base_in, psi_base_ex, psi_base_in):
    H_base_ex = 1.0
    H_base_in = 0.7
    return L_base_ex * psi_base_ex * H_base_ex + L_base_in * psi_base_in * H_base_in

### 3.4 単位日射強度当たりの暖房期・冷房期の日射熱取得量

#### 1) 合計

$$
m_{H,total} = m_{H,roof} + m_{H,window} + m_{H,wall}
$$

$$
m_{C,total} = m_{C,roof} + m_{C,window} + m_{C,wall}
$$

$m_{H,total}$：単位日射強度当たりの暖房期の日射熱取得量(W/(W/m<sup>2</sup>))  
$m_{C,total}$：単位日射強度当たりの冷房期の日射熱取得量(W/(W/m<sup>2</sup>))  
$m_{H,roof}$：天井又は外気屋根の単位日射強度当たりの暖房期の日射熱取得量(W/(W/m<sup>2</sup>))  
$m_{C,roof}$：天井又は外気屋根の単位日射強度当たりの冷房期の日射熱取得量(W/(W/m<sup>2</sup>))  
$m_{H,window}$：窓の単位日射強度当たりの暖房期の日射熱取得量(W/(W/m<sup>2</sup>))  
$m_{C,window}$：窓の単位日射強度当たりの冷房期の日射熱取得量(W/(W/m<sup>2</sup>))  
$m_{H,wall}$：壁の単位日射強度当たりの暖房期の日射熱取得量(W/(W/m<sup>2</sup>))  
$m_{C,wall}$：壁の単位日射強度当たりの冷房期の日射熱取得量(W/(W/m<sup>2</sup>))  

In [12]:
def get_m_H_total(m_H_roof, m_H_window, m_H_wall):
    return m_H_roof + m_H_window + m_H_wall

In [13]:
def get_m_C_total(m_C_roof, m_C_window, m_C_wall):
    return m_C_roof + m_C_window + m_C_wall

#### 2) 天井又は外気屋根

$$
m_{H,roof} = \sum_{i=1} \left ( A_{roof,i} \times \eta_{H,roof} \times \nu_{H,roof} \right )
$$

$$
m_{C,roof} = \sum_{i=1} \left ( A_{roof,i} \times \eta_{C,roof} \times \nu_{C,roof} \right )
$$

$A_{roof,i}$：i階の天井又は外気屋根の面積の合計(m<sup>2</sup>)  
$\eta_{H,roof}$：天井又は外気屋根の暖房期の日射熱取得率((W/m<sup>2</sup>)/(W/m<sup>2</sup>))  
$\eta_{C,roof}$：天井又は外気屋根の冷房期の日射熱取得率((W/m<sup>2</sup>)/(W/m<sup>2</sup>))  
$\nu_{H,roof}$：天井又は外気屋根の暖房期の方位係数  
$\nu_{C,roof}$：天井又は外気屋根の冷房期の方位係数  

In [14]:
def get_m_H_roof(A_roof, eta_H_roof, nu_H_roof):
    return sum( [ A_roof_i * eta_H_roof * nu_H_roof for A_roof_i in A_roof ] )

In [15]:
def get_m_C_roof(A_roof, eta_C_roof, nu_C_roof):
    return sum( [ A_roof_i * eta_C_roof * nu_C_roof for A_roof_i in A_roof ] )

#### 3) 窓

$$
m_{H,window} = A_{window,0} \times \eta_{H,window,0} \times \nu_{H,0} + A_{window,90} \times \eta_{H,window,90} \times \nu_{H,90} + A_{window,180} \times \eta_{H,window,180} \times \nu_{H,180} + A_{window,270} \times \eta_{H,window,270} \times \nu_{H,270}
$$

$$
\eta_{H,window,0} = \eta_{H,window,main}
$$

$$
\eta_{H,window,90} = \eta_{H,window,other}
$$

$$
\eta_{H,window,180} = \eta_{H,window,other}
$$

$$
\eta_{H,window,270} = \eta_{H,window,other}
$$

$$
m_{C,window} = A_{window,0} \times \eta_{C,window,0} \times \nu_{C,0} + A_{window,90} \times \eta_{C,window,90} \times \nu_{C,90} + A_{window,180} \times \eta_{C,window,180} \times \nu_{C,180} + A_{window,270} \times \eta_{C,window,270} \times \nu_{C,270}
$$

$$
\eta_{C,window,0} = \eta_{C,window,main}
$$

$$
\eta_{C,window,90} = \eta_{C,window,other}
$$

$$
\eta_{C,window,180} = \eta_{C,window,other}
$$

$$
\eta_{C,window,270} = \eta_{C,window,other}
$$

$A_{window,0}$：主開口方位の窓面積の合計(m<sup>2</sup>)  
$A_{window,90}$：主開口方位から時計回りに90度の方位の窓面積の合計(m<sup>2</sup>)  
$A_{window,180}$：主開口方位から時計回りに180度の方位の窓面積の合計(m<sup>2</sup>)  
$A_{window,270}$：主開口方位から時計回りに270度の方位の窓面積の合計(m<sup>2</sup>)  
$\eta_{H,window,0}$：主開口方位の窓の暖房期の日射熱取得率((W/m<sup>2</sup>)/(W/m<sup>2</sup>))  
$\eta_{H,window,90}$：主開口方位から時計回りに90度の方位の窓の暖房期の日射熱取得率((W/m<sup>2</sup>)/(W/m<sup>2</sup>))  
$\eta_{H,window,180}$：主開口方位から時計回りに180度の方位の窓の暖房期の日射熱取得率((W/m<sup>2</sup>)/(W/m<sup>2</sup>))  
$\eta_{H,window,270}$：主開口方位から時計回りに270度の方位の窓の暖房期の日射熱取得率((W/m<sup>2</sup>)/(W/m<sup>2</sup>))  
$\eta_{C,window,0}$：主開口方位の窓の冷房期の日射熱取得率((W/m<sup>2</sup>)/(W/m<sup>2</sup>))  
$\eta_{C,window,90}$：主開口方位から時計回りに90度の方位の窓の冷房期の日射熱取得率((W/m<sup>2</sup>)/(W/m<sup>2</sup>))  
$\eta_{C,window,180}$：主開口方位から時計回りに180度の方位の窓の冷房期の日射熱取得率((W/m<sup>2</sup>)/(W/m<sup>2</sup>))  
$\eta_{C,window,270}$：主開口方位から時計回りに270度の方位の窓の冷房期の日射熱取得率((W/m<sup>2</sup>)/(W/m<sup>2</sup>))  
$\eta_{H,window,main}$：主開口方位の窓の暖房期の日射熱取得率  
$\eta_{H,window,other}$：主開口方位以外の窓の暖房期の日射熱取得率  
$\eta_{C,window,main}$：主開口方位の窓の冷房期の日射熱取得率  
$\eta_{C,window,other}$：主開口方位以外の窓の冷房期の日射熱取得率  
$\nu_{H,0}$：主開口方位の暖房期の方位係数  
$\nu_{H,90}$：主開口方位から時計回りに90度の方位の暖房期の方位係数  
$\nu_{H,180}$：主開口方位から時計回りに180度の方位の暖房期の方位係数  
$\nu_{H,270}$：主開口方位から時計回りに270度の方位の暖房期の方位係数  
$\nu_{C,0}$：主開口方位の冷房期の方位係数  
$\nu_{C,90}$：主開口方位から時計回りに90度の方位の冷房期の方位係数  
$\nu_{C,180}$：主開口方位から時計回りに180度の方位の冷房期の方位係数  
$\nu_{C,270}$：主開口方位から時計回りに270度の方位の冷房期の方位係数  

In [16]:
def get_m_H_window(A_window, eta_H_window_main, eta_H_window_other, nu_H):
    eta_H_window = Orientation( deg0   = eta_H_window_main,
                                deg90  = eta_H_window_other,
                                deg180 = eta_H_window_other,
                                deg270 = eta_H_window_other )
    return   A_window.deg0   * eta_H_window.deg0   * nu_H.deg0 \
           + A_window.deg90  * eta_H_window.deg90  * nu_H.deg90 \
           + A_window.deg180 * eta_H_window.deg180 * nu_H.deg180 \
           + A_window.deg270 * eta_H_window.deg270 * nu_H.deg270 

In [17]:
def get_m_C_window(A_window, eta_C_window_main, eta_C_window_other, nu_C):
    eta_C_window = Orientation( deg0   = eta_C_window_main,
                                deg90  = eta_C_window_other,
                                deg180 = eta_C_window_other,
                                deg270 = eta_C_window_other )
    return   A_window.deg0   * eta_C_window.deg0   * nu_C.deg0 \
           + A_window.deg90  * eta_C_window.deg90  * nu_C.deg90 \
           + A_window.deg180 * eta_C_window.deg180 * nu_C.deg180 \
           + A_window.deg270 * eta_C_window.deg270 * nu_C.deg270 

#### 4) 壁

$$
m_{H,wall} = A_{wall,0} \times \eta_{H,wall} \times \nu_{H,0} + A_{wall,90} \times \eta_{H,wall} \times \nu_{H,90} + A_{wall,180} \times \eta_{H,wall} \times \nu_{H,180} + A_{wall,270} \times \eta_{H,wall} \times \nu_{H,270}
$$

$$
m_{C,wall} = A_{wall,0} \times \eta_{C,wall} \times \nu_{C,0} + A_{wall,90} \times \eta_{C,wall} \times \nu_{C,90} + A_{wall,180} \times \eta_{C,wall} \times \nu_{C,180} + A_{wall,270} \times \eta_{C,wall} \times \nu_{C,270}
$$

$A_{wall,0}$：主開口方位の壁面積の合計(m<sup>2</sup>)  
$A_{wall,90}$：主開口方位から時計回りに90度の方位の壁面積の合計(m<sup>2</sup>)  
$A_{wall,180}$：主開口方位から時計回りに180度の方位の壁面積の合計(m<sup>2</sup>)  
$A_{wall,270}$：主開口方位から時計回りに270度の方位の壁面積の合計(m<sup>2</sup>)  
$\eta_{H,wall}$：壁の暖房期の日射熱取得率((W/m<sup>2</sup>K)/(W/m<sup>2</sup>K))  
$\eta_{C,wall}$：壁の冷房期の日射熱取得率((W/m<sup>2</sup>K)/(W/m<sup>2</sup>K))  
$\nu_{H,0}$：主開口方位の暖房期の方位係数  
$\nu_{H,90}$：主開口方位から時計回りに90度の方位の暖房期の方位係数  
$\nu_{H,180}$：主開口方位から時計回りに180度の方位の暖房期の方位係数  
$\nu_{H,270}$：主開口方位から時計回りに270度の方位の暖房期の方位係数
$\nu_{C,0}$：主開口方位の冷房期の方位係数  
$\nu_{C,90}$：主開口方位から時計回りに90度の方位の冷房期の方位係数  
$\nu_{C,180}$：主開口方位から時計回りに180度の方位の冷房期の方位係数  
$\nu_{C,270}$：主開口方位から時計回りに270度の方位の冷房期の方位係数

In [18]:
def get_m_H_wall(A_wall, eta_H_wall, nu_H):
    return   A_wall.deg0   * eta_H_wall * nu_H.deg0 \
           + A_wall.deg90  * eta_H_wall * nu_H.deg90 \
           + A_wall.deg180 * eta_H_wall * nu_H.deg180 \
           + A_wall.deg270 * eta_H_wall * nu_H.deg270    

In [19]:
def get_m_C_wall(A_wall, eta_C_wall, nu_C):
    return   A_wall.deg0   * eta_C_wall * nu_C.deg0 \
           + A_wall.deg90  * eta_C_wall * nu_C.deg90 \
           + A_wall.deg180 * eta_C_wall * nu_C.deg180 \
           + A_wall.deg270 * eta_C_wall * nu_C.deg270    

### 3.5 日射熱取得率(開口部を除く)

#### 1) 天井又は外気屋根

$$
\eta_{H,roof} = U_{roof} \times 0.034
$$

$$
\eta_{C,roof} = U_{roof} \times 0.034
$$

$\eta_{H,roof}$：天井又は外気屋根の暖房期の日射熱取得率((W/m<sup>2</sup>)/(W/m<sup>2</sup>))  
$\eta_{C,roof}$：天井又は外気屋根の冷房期の日射熱取得率((W/m<sup>2</sup>)/(W/m<sup>2</sup>))  
$U_{roof}$：天井又は外気屋根の熱貫流率(W/m<sup>2</sup>K)  

In [20]:
def get_eta_H_roof(U_roof):
    return U_roof * 0.034

In [21]:
def get_eta_C_roof(U_roof):
    return U_roof * 0.034

#### 2) 壁

$$
\eta_{H,wall} = U_{wall} \times 0.034
$$

$$
\eta_{C,wall} = U_{wall} \times 0.034
$$

$\eta_{H,wall}$：壁の暖房期の日射熱取得率((W/m<sup>2</sup>K)/(W/m<sup>2</sup>K))  
$\eta_{C,wall}$：壁の冷房期の日射熱取得率((W/m<sup>2</sup>K)/(W/m<sup>2</sup>K))  
$U_{wall}$：壁の熱貫流率(W/m<sup>2</sup>K)  

In [22]:
def get_eta_H_wall(U_wall):
    return U_wall * 0.034

In [23]:
def get_eta_C_wall(U_wall):
    return U_wall * 0.034

### 3.6 方位係数

#### 1) 上側

In [24]:
def get_nu_H_top():
    return 1.0

In [25]:
def get_nu_C_top():
    return 1.0

#### 2) 水平方向

In [26]:
def get_orientation(main_direction):
    # main_direction : 'S','SW','W','NW','N','NE','E',SE' as string
    return {
        'S'  : ['S',  'W',  'N',  'E'  ], 
        'SW' : ['SW', 'NW', 'NE', 'SE' ],
        'W'  : ['W',  'N',  'E',  'S'  ],
        'NW' : ['NW', 'NE', 'SE', 'SW' ],
        'N'  : ['N',  'E',  'S',  'W'  ],
        'NE' : ['NE', 'SE', 'SW', 'NW' ],
        'E'  : ['E',  'S',  'W',  'N'  ],
        'SE' : ['SE', 'SW', 'NW', 'NE' ]
    }[main_direction]

In [27]:
def get_nu_H_horizontal(region, main_direction):
    # main_direction : 'S','SW','W','NW','N','NE','E',SE' as string
    return Orientation._make( [ DC_get_nu_H(region)[ort] for ort in get_orientation(main_direction) ] )

In [28]:
def get_nu_C_horizontal(region, main_direction):
    # main_direction : 'S','SW','W','NW','N','NE','E',SE' as string
    return Orientation._make( [ DC_get_nu_C(region)[ort] for ort in get_orientation(main_direction) ] )

#### Example

In [29]:
get_nu_H_top()

1.0

In [30]:
get_nu_C_top()

1.0

In [31]:
get_nu_H_horizontal('region1', 'S')

Orientation(deg0=0.935, deg90=0.535, deg180=0.26, deg270=0.564)

In [32]:
get_nu_C_horizontal('region1', 'S')

Orientation(deg0=0.502, deg90=0.508, deg180=0.329, deg270=0.545)

### 3.7 面積 

#### 1) 合計

$$
\begin{array}{l}
A_{env,total} &= \sum_{i=1} A_{roof,i} \\
&+ A_{window,0} + A_{window,90} + A_{window,180} + A_{window,270} \\
&+ A_{wall,0} + A_{wall,90} + A_{wall,180} + A_{wall,270} \\
&+ \sum_{i=1} A_{exfloor,i} + A_{EF}
\end{array}
$$

$A_{env,total}$：部位の面積の合計(m<sup>2</sup>)  
$A_{roof,i}$：i階の天井又は外気屋根の面積の合計(m<sup>2</sup>)  
$A_{window,0}$：主開口方位の窓面積の合計(m<sup>2</sup>)  
$A_{window,90}$：主開口方位から時計回りに90度の方位の窓面積の合計(m<sup>2</sup>)  
$A_{window,180}$：主開口方位から時計回りに180度の方位の窓面積の合計(m<sup>2</sup>)  
$A_{window,270}$：主開口方位から時計回りに270度の方位の窓面積の合計(m<sup>2</sup>)  
$A_{wall,0}$：主開口方位の壁面積の合計(m<sup>2</sup>)  
$A_{wall,90}$：主開口方位から時計回りに90度の方位の壁面積の合計(m<sup>2</sup>)  
$A_{wall,180}$：主開口方位から時計回りに180度の方位の壁面積の合計(m<sup>2</sup>)  
$A_{wall,270}$：主開口方位から時計回りに270度の方位の壁面積の合計(m<sup>2</sup>)  
$A_{exfloor,i}$：i階の外気床の面積の合計(m<sup>2</sup>)  
$A_{EF}$：土間床の面積の合計(m<sup>2</sup>)  

In [33]:
def get_A_env_total(A_roof, A_window, A_wall, A_exfloor, A_EF):
    return   sum(A_roof) + sum(A_window) + sum(A_wall) + sum(A_exfloor) + A_EF

#### 2) 天井又は外気屋根

天井又は外気屋根の面積の算出において、屋根断熱の場合は屋根の勾配を考慮して計算する。

$i \ne N$の場合($N$・・・階数)

$$
A_{roof,i} = \left \{
\begin{array}{ll}
    0 & (A_{floor,i} \le A_{floor,i+1}) \\
    (A_{floor,i} - A_{floor,i+1}) \times \sqrt{\left(\frac{D_{pitch,i}}{10}\right)^2 + 1} & ((A_{floor,i} \gt A_{floor,i+1})\text{&屋根断熱}) \\
    A_{floor,i} - A_{floor,i+1} & ((A_{floor,i} \gt A_{floor,i+1})\text{&天井断熱})
\end{array}
\right.
$$

$i=N$の場合(最上階の場合)

$$
A_{roof,i} = \left \{
\begin{array}{ll}
    A_{floor,i} \times \sqrt{\left(\frac{D_{pitch,i}}{10}\right)^2 + 1} & (\text{屋根断熱}) \\
    A_{floor,i} & (\text{天井断熱})
\end{array}
\right.
$$

$A_{floor,i}$：i階部分の床面積の合計(m<sup>2</sup>)  
$D_{pitch,i}$：i階外気屋根の勾配(寸)  

In [34]:
def get_A_roof(A_floor, D_pitch, roof_ins_type):
    # roof_ins_type : 'roof' or 'ceiling' as string

    def get_A_roof_i(A_floor_i, A_floor_i_plus, D_pitch_i, roof_ins_type):
        return {
            'roof'    : max( (A_floor_i - A_floor_i_plus), 0.0 ) * math.sqrt((D_pitch_i/10)**2 + 1),
            'ceiling' : max( (A_floor_i - A_floor_i_plus), 0.0 )
        }[roof_ins_type]
        
    def get_A_roof_top(A_floor_i, D_pitch_i, roof_ins_type):
        return {
            'roof'    : A_floor_i * math.sqrt((D_pitch_i/10)**2 + 1),
            'ceiling' : A_floor_i
        }[roof_ins_type]

    A_roof = []
    
    for i in range(len(A_floor)):
        if i == len(A_floor) - 1:
            A_roof.append( get_A_roof_top(A_floor[i], D_pitch[i], roof_ins_type) )
        else:
            A_roof.append( get_A_roof_i(A_floor[i], A_floor[i+1], D_pitch[i], roof_ins_type) )

    return A_roof

#### 3) 窓

$$
A_{window,0} = A_{window,total} \times r_0
$$

$$
A_{window,90} = A_{window,total} \times r_{90}
$$

$$
A_{window,180} = A_{window,total} \times r_{180}
$$

$$
A_{window,270} = A_{window,total} \times r_{270}
$$

$A_{window,total}$：窓面積の合計(m<sup>2</sup>)  
$r_0$,$r_{90}$,$r_{180}$,$r_{270}$：主開口方位から時計回りに0度、90度、180度、270度の方位の窓面積の比率

$r_0$,$r_{90}$,$r_{180}$,$r_{270}$は、次に示す自立循環型住宅モデルの窓面積をもとに比率を算出する。

| 方位 | 面積(m<sup>2</sup>) |
| :---: | :---: |
| 主方位 | 19.7 |
| 主方位+90° | 3.96 |
| 主方位+180° | 4.77 |
| 主方位+270° | 3.79 |
| 合計 | 32.22 |


In [35]:
def get_A_window(A_window_total):
    rs = [19.7, 3.96, 4.77, 3.79]
    return Orientation._make([ A_window_total * r / sum(rs) for r in rs ])

#### 4) 壁

$$
A_{wall,0} = \sum_{i=1} \left( L_{long,i} \times h_i \right) - A_{window,0}
$$

$$
A_{wall,90} = \sum_{i=1} \left( L_{short,i} \times h_i \right) - A_{window,90}
$$

$$
A_{wall,180} = \sum_{i=1} \left( L_{long,i} \times h_i \right) - A_{window,180}
$$

$$
A_{wall,270} = \sum_{i=1} \left( L_{short,i} \times h_i \right) - A_{window,270}
$$

$L_{long,i}$：$i$階の長辺側の壁幅(m)  
$L_{short,i}$：$i$階の短辺側の壁幅(m)  
$h_i$：$i$階の高さ(m)  

In [36]:
def get_A_wall(L_long, L_short, h, A_window):
    A_wall_0   = sum( [ L_i * h_i for L_i,h_i in zip(L_long,  h) ] ) - A_window.deg0
    A_wall_90  = sum( [ L_i * h_i for L_i,h_i in zip(L_short, h) ] ) - A_window.deg90
    A_wall_180 = sum( [ L_i * h_i for L_i,h_i in zip(L_long,  h) ] ) - A_window.deg180
    A_wall_270 = sum( [ L_i * h_i for L_i,h_i in zip(L_short, h) ] ) - A_window.deg270
    return Orientation(A_wall_0, A_wall_90, A_wall_180, A_wall_270)

#### 5) 外気床

$i=1$の場合(1階の場合)

$$
A_{exfloor,i} = A_{floor,i} - A_{EF}
$$

$i \ne 1$の場合

$$
A_{exfloor,i} = \left \{
\begin{array}{ll}
    0 & (A_{floor,i} \le A_{floor,i-1}) \\
    A_{floor,i} - A_{floor,i-1} & (A_{floor,i} \gt A_{floor,i-1}) \\
\end{array}
\right.
$$

In [37]:
def get_A_exfloor(A_floor, A_EF):

    A_exfloor = []
    
    for i in range(len(A_floor)):
        if i == 0:
            A_exfloor.append(A_floor[i] - A_EF)
        else:
            A_exfloor.append( A_floor[i] - A_floor[i-1] if A_floor[i] > A_floor[i-1] else 0.0 )
    
    return A_exfloor

#### 6) 土間床

土間床の面積は床断熱の場合は自立循環型住宅モデルの土間床と外気床の比率をもとに計算する。

$$
A_{EF} = \left \{
\begin{array}{ll}
    A_{floor,1} * r_{EF} & (\text{床断熱}) \\
    A_{floor,1} & (\text{基礎断熱})
\end{array}
\right.
$$

$r_{EF}$：床断熱時の1階床面積(土間床面積と1階の外気床の面積の合計)に対する土間床面積の比

土間床面積と1階の外気床の面積は次のとおりとする。

| 種類 | 面積(m<sup>2</sup>) | 比 |
| :---: | :---: | :---: |
| １階床 | 65.42 | 0.963 |
| 土間 | 2.48 | 0.037 |


※「比」は、小数点第三位までで四捨五入した値。

In [38]:
def get_A_EF(A_floor_1, floor_ins_type):
    # floor_ins_type : 'base', 'floor' as string
    return {
        'base'  : A_floor_1,
        'floor' : A_floor_1 * 0.037
    }[floor_ins_type]

### 3.8 長さ

#### 1) 壁幅

$i$階の外周長を$L_{p,i}$、$i$階の床面積の合計を$A_{floor,i}$とおき、$i$階の長辺側・短辺側の壁幅をそれぞれ、$L_{long,i}$、$L_{short,i}$とすると次が成り立つ。

$$
L_{p,i} = ( L_{long,i} + L_{short,i} ) \times 2
$$

$$
A_{floor,i} = L_{long,i} \times L_{short,i}
$$

両式から、$L_{short,i}$を消すと、

$$
A_{floor,i} = L_{long,i} \times \left( \frac{L_{p,i}}{2} - L_{long,i} \right)
$$

$$
\Leftrightarrow L_{long,i}^2 - \frac{L_{p,i}}{2} \times L_{long,i} + A_{floor,i} = 0
$$

となり、

$$
L_{long,i} = \frac{ \frac{L_{p,i}}{2} + \sqrt{ \left ( \frac{L_{p,i}}{2} \right ) ^2 - 4 \times A_{floor,i} } }{2}
$$

$$
L_{short,i} = \frac{A_{floor,i}}{L_{long,i}}
$$

$L_{long,i}$：i階の長辺側の壁幅(m)  
$L_{short,i}$：i階の短辺側の壁幅(m)  
$L_{p,i}$：i階の外周長(m)  
$A_{floor,i}$：i階の床面積の合計(m<sup>2</sup>)  

In [39]:
def get_L_long_i(L_p_i,A_floor_i):
    return ( L_p_i/2 + math.sqrt( (L_p_i/2)**2 - 4*A_floor_i  ) )/2

In [40]:
def get_L_short_i(L_p_i,A_floor_i):
    return A_floor_i / get_L_long_i(L_p_i,A_floor_i)

#### 2) 土間床外周部及び基礎等の長さ

$$
L_{F,ex} = \left \{
\begin{array}{ll}
    \sqrt{ A_{EF} } \times 2 & (床断熱) \\
    L_{p,1} & (基礎断熱)
\end{array}
\right.
$$

$$
L_{F,in} = \left \{
\begin{array}{ll}
    \sqrt{A_{EF}} \times 2 & (床断熱) \\
    0 & (基礎断熱)
\end{array}
\right.
$$

$L_{F,ex}$：土間床外周部及び基礎等の外気側の長さ(m)  
$L_{F,in}$：土間床外周部及び基礎等の室内側の長さ(m)  
$L_{p,1}$：1階の外周長(m)  
$A_{EF}$：土間床の面積の合計(m<sup>2</sup>)

In [41]:
def get_L_F_ex(A_EF, L_p_1, floor_ins_type):
    # floor_ins_type : 'base', 'floor' as string
    return {
        'floor' : math.sqrt(A_EF) * 2,
        'base'  : L_p_1
    }[floor_ins_type]

In [42]:
def get_L_F_in(A_EF, floor_ins_type):
    # floor_ins_type : 'base', 'floor' as string
    return {
        'floor' : math.sqrt(A_EF) * 2,
        'base'  : 0.0
    }[floor_ins_type]

#### 3) 高さ

最上階でかつ屋根断熱の場合は、屋根勾配がある分だけ階高にプラスして高さを計算することとする。屋根勾配は安全側の数値として、長辺側の壁幅をもとに計算する。屋根形状は両流れ屋根とするため2で割っている。

$$
h_i = \left \{
\begin{array}{ll}
h_{story,i} + L_{long,i} \times \frac{ D_{pitch,i} }{10} \times \frac{1}{2} & (\text{屋根断熱 & 最上階}(i=N)) \\
h_{story,i} & (\text{上記以外})
\end{array}
\right.
$$

In [43]:
def get_h(h_story, L_long, D_pitch, roof_ins_type, nstory):
    # roof_ins_type : 'roof' 'ceiling' as string
    # nstory : number of story, '1st' '2nd' '3rd' as string
    
    h = h_story
    
    i = { '1st' : 0, '2nd' : 1, '3rd' : 2 }
    
    if roof_ins_type == 'roof':
        h[i] = h[i] + L_long[i] * D_pitch[i] / 10 / 2
        
    return h

#### 4) 階高

階高を指定しない場合は、1階を2.9m、2階を2.7m、3階を2.7mとする。  
階高を指定する場合は、建物高さを階数でわって計算する。

In [44]:
def get_h_story(nstory, isSpecifyHeight, h):
    # nstory : number of story, '1st' '2nd' '3rd' as string
    # isSpecifyHeight : 'yes' 'no' as string
    # h : total height of building (m)
    return {
        'yes' : {
            '1st' : [h],
            '2nd' : [h/2, h/2],
            '3rd' : [h/3, h/3, h/3]
        }[nstory],
        'no'  : {
            '1st' : [2.9],
            '2nd' : [2.9, 2.7],
            '3rd' : [2.9, 2.7, 2.7]
        }[nstory],
    }[isSpecifyHeight]

#### 使用例

In [45]:
get_h_story('1st','yes',3.0)

[3.0]

In [46]:
get_h_story('1st','no',3.0)

[2.9]

In [47]:
get_h_story('2nd','yes',6.0)

[3.0, 3.0]

In [48]:
get_h_story('2nd','no',6.0)

[2.9, 2.7]

In [49]:
get_h_story('3rd','yes',9.0)

[3.0, 3.0, 3.0]

In [50]:
get_h_story('3rd','no',9.0)

[2.9, 2.7, 2.7]

## 4. U<sub>A</sub>値・η<sub>A</sub>値の計算

In [51]:
def get_env_performance(spec, isDEBUG):
    region             = spec['region']
    roof_ins_type      = spec['roof_ins_type']
    floor_ins_type     = spec['floor_ins_type']
    nstory             = spec['nstory']
    isSpecifyHeight    = spec['isSpecifyHeight']
    h_total            = spec['total_height']
    main_direction     = spec['main_direction']
    A_floor            = spec['floor_area']
    L_p                = spec['perimeter_length']
    A_window_total     = spec['total_wnd_area']
    U_wall             = spec['U_wall']
    U_roof             = spec['U_roof']
    U_exfloor_C        = spec['U_exfloor_C']
    U_exfloor_O        = spec['U_exfloor_O']
    psi_base_ex        = spec['psi_fund_ex']
    psi_base_in        = spec['psi_fund_in']
    U_window_main      = spec['U_wnd_main']
    U_window_other     = spec['U_wnd_other']
    eta_H_window_main  = spec['eta_H_wnd_main']
    eta_H_window_other = spec['eta_H_wnd_other']
    eta_C_window_main  = spec['eta_C_wnd_main']
    eta_C_window_other = spec['eta_C_wnd_other']
    D_pitch            = spec['D_pitch']

    h_story = get_h_story(nstory, isSpecifyHeight, h_total)
    if isDEBUG: print( '階高(m):' + str(h_story))
        
    L_long = [ get_L_long_i(l,a) for l,a in zip(L_p, A_floor) ]
    if isDEBUG: print( '長辺側の壁幅(m):' + str(L_long))
    
    L_short = [ get_L_short_i(l,a) for l,a in zip(L_p, A_floor) ]
    if isDEBUG: print( '短辺側の壁幅(m):' + str(L_short) )
    
    h = get_h(h_story, L_long, D_pitch, roof_ins_type, nstory)
    if isDEBUG: print( '高さ(m):' + str(h) )
    
    A_EF = get_A_EF(A_floor[0], floor_ins_type)
    if isDEBUG: print( '土間床の面積の合計(m):' + str(A_EF))
    
    L_base_ex = get_L_F_ex(A_EF, L_p[0], floor_ins_type)
    if isDEBUG: print('土間床外周部及び基礎等の外気側の長さ(m):' + str(L_base_ex))
        
    L_base_in = get_L_F_in(A_EF, floor_ins_type)
    if isDEBUG: print('土間床外周部及び基礎等の室内側の長さ(m):' + str(L_base_in))
    
    A_exfloor = get_A_exfloor(A_floor, A_EF)
    if isDEBUG: print('外気床の面積の合計(m2):' + str(A_exfloor))
    
    A_window = get_A_window(A_window_total)
    if isDEBUG: print('窓面積の合計(m2):' + str(A_window))
        
    A_wall = get_A_wall(L_long, L_short, h, A_window)
    if isDEBUG: print('壁面積の合計(m2):' + str(A_wall))
        
    A_roof = get_A_roof(A_floor, D_pitch, roof_ins_type)
    if isDEBUG: print('天井又は外気屋根の面積の合計(m2):' + str(A_roof))

    A_env_total = get_A_env_total(A_roof, A_window, A_wall, A_exfloor, A_EF)
    if isDEBUG: print('部位の面積の合計(m2):' + str(A_env_total))
    
    q_roof = get_q_roof(A_roof, U_roof)
    if isDEBUG: print('q値(天井又は外気屋根)(W/K):' + str(q_roof))
    
    q_window = get_q_window(A_window, U_window_main, U_window_other)
    if isDEBUG: print('q値(窓)(W/K):' + str(q_window))
    
    q_wall = get_q_wall(A_wall, U_wall)
    if isDEBUG: print('q値(壁)(W/K):' + str(q_wall))
    
    q_floor = get_q_floor(A_exfloor, U_exfloor_C, U_exfloor_O)
    if isDEBUG: print('q値(外気床)(W/K):' + str(q_floor))
    
    q_base = get_q_base(L_base_ex, L_base_in, psi_base_ex, psi_base_in)
    if isDEBUG: print('q値(土間床外周部及び基礎等)(W/K):' + str(q_base))
    
    q_total = get_q_total(q_roof, q_window, q_wall, q_floor, q_base)
    if isDEBUG: print('q値(合計)(W/K):' + str(q_total))
    
    U_A = get_U_A(q_total, A_env_total)
    if isDEBUG: print('UA値(W/m2K):' + str(U_A))
    
    nu_H_top = get_nu_H_top()
    nu_C_top = get_nu_C_top()
    if isDEBUG: print('方位係数(上側) 暖房期:' + str(nu_H_top) + ',冷房期:' + str(nu_C_top) )
        
    nu_H_horizontal = get_nu_H_horizontal(region, main_direction)
    nu_C_horizontal = get_nu_C_horizontal(region, main_direction)
    if isDEBUG: print('方位係数(水平方向) 暖房期:' + str(nu_H_horizontal) + ',冷房期:' + str(nu_C_horizontal) )

    eta_H_roof = get_eta_H_roof(U_roof)
    eta_C_roof = get_eta_C_roof(U_roof)
    if isDEBUG: print('η値(屋根) 暖房期:' + str(eta_H_roof) + ',冷房期:' + str(eta_C_roof) )

    eta_H_wall = get_eta_H_wall(U_wall)
    eta_C_wall = get_eta_C_wall(U_wall)
    if isDEBUG: print('η値(壁) 暖房期:' + str(eta_H_wall) + ',冷房期:' + str(eta_C_wall) )

    m_H_roof = get_m_H_roof(A_roof, eta_H_roof, nu_H_top)
    m_C_roof = get_m_C_roof(A_roof, eta_C_roof, nu_C_top)
    if isDEBUG: print('m値(屋根) 暖房期:' + str(m_H_roof) + ',冷房期:' + str(m_C_roof) )

    m_H_window = get_m_H_window(A_window, eta_H_window_main, eta_H_window_other, nu_H_horizontal)
    m_C_window = get_m_C_window(A_window, eta_C_window_main, eta_C_window_other, nu_C_horizontal)
    if isDEBUG: print('m値(窓) 暖房期:' + str(m_H_window) + ',冷房期:' + str(m_C_window) )

    m_H_wall = get_m_H_wall(A_wall, eta_H_wall, nu_H_horizontal)
    m_C_wall = get_m_C_wall(A_wall, eta_C_wall, nu_C_horizontal)
    if isDEBUG: print('m値(壁) 暖房期:' + str(m_H_wall) + ',冷房期:' + str(m_C_wall) )

    m_H_total = get_m_H_total(m_H_roof, m_H_window, m_H_wall)
    m_C_total = get_m_C_total(m_C_roof, m_C_window, m_C_wall)
    if isDEBUG: print('m値(合計) 暖房期:' + str(m_H_total) + ',冷房期:' + str(m_C_total) )

    eta_A_H = get_eta_A_H(m_H_total, A_env_total)
    eta_A_C = get_eta_A_C(m_C_total, A_env_total)
    if isDEBUG: print('ηA値 暖房期:' + str(eta_A_H) + ',冷房期:' + str(eta_A_C) )
    
    return (U_A, eta_A_H, eta_A_C)

#### Example

In [52]:
spec = {
    'region'          : 'region6',
    'roof_ins_type'   : 'ceiling',
    'floor_ins_type'  : 'floor',
    'nstory'          : '2nd',
    'isSpecifyHeight' : 'yes',
    'total_height'    : 5.6,
    'main_direction'  : 'S',
    'floor_area'      : [ 67.89, 52.19 ],
    'perimeter_length': [ 35.5948077112379, 30.0529410208053 ],
    'total_wnd_area'  : 32.28,
    'U_wall'          : 0.53,
    'U_roof'          : 0.24,
    'U_exfloor_C'     : 0.48,
    'U_exfloor_O'     : 0.34,
    'psi_fund_ex'     : 1.8,
    'psi_fund_in'     : 1.8,
    'U_wnd_main'      : 4.65,
    'U_wnd_other'     : 4.65,
    'eta_H_wnd_main'  : 0.74 * 0.51,
    'eta_H_wnd_other' : 0.74 * 0.51,
    'eta_C_wnd_main'  : 0.74 * 0.93,
    'eta_C_wnd_other' : 0.74 * 0.93,
    'D_pitch'         : [ 0.0, 0.0]
}
get_env_performance(spec,True)

階高(m):[2.8, 2.8]
長辺側の壁幅(m):[12.259787466738834, 9.576898018348473]
短辺側の壁幅(m):[5.5376163888801155, 5.449572492054177]
高さ(m):[2.8, 2.8]
土間床の面積の合計(m):2.51193
土間床外周部及び基礎等の外気側の長さ(m):3.169813874662044
土間床外周部及び基礎等の室内側の長さ(m):3.169813874662044
外気床の面積の合計(m2):[65.37807, 0.0]
窓面積の合計(m2):Orientation(deg0=19.736685288640597, deg90=3.967374301675978, deg180=4.778882681564245, deg270=3.7970577281191806)
壁面積の合計(m2):Orientation(deg0=41.40603406960385, deg90=26.796754564940038, deg180=56.3638366766802, deg270=26.967071138496834)
天井又は外気屋根の面積の合計(m2):[15.700000000000003, 52.19]
部位の面積の合計(m2):319.5936964497209
q値(天井又は外気屋根)(W/K):16.293599999999998
q値(窓)(W/K):150.10200000000003
q値(壁)(W/K):80.3128591183521
q値(外気床)(W/K):21.967031519999995
q値(土間床外周部及び基礎等)(W/K):9.699630456465854
q値(合計)(W/K):278.375121094818
UA値(W/m2K):0.8710281966985307
方位係数(上側) 暖房期:1.0,冷房期:1.0
方位係数(水平方向) 暖房期:Orientation(deg0=0.936, deg90=0.523, deg180=0.261, deg270=0.579),冷房期:Orientation(deg0=0.434, deg90=0.504, deg180=0.341, deg270=0.512)
η値(屋根) 

(0.8710281966985307, 3.4752873446464307, 3.581661971625305)